# scRNAseq Random Forest Model Evaluation - GSE154567

In [1]:
import joblib
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle
import scanpy as sc
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report as class_rep
import time

In [2]:
# Prep
dataset_name = 'GSE154567' # label for the dataset
dictionary_dir = 'RFSplitDicts' # dir where we save the split dictionaries
model_dir = 'Model_Splits' # dir where we save the models

In [3]:
# Make sure necessary directories are avalible

# dictionary dir
if not os.path.exists(dictionary_dir):
    os.makedirs(dictionary_dir)
    print(f"Directory {dictionary_dir} created for saving split dictionaries")
    
else:
    print('Directory already exists!')


# model dir
if not os.path.exists(model_dir):
    os.makedirs(model_dir)
    print(f"Directory {model_dir} created for saving the trained models for the splits")
    
else:
    print('Directory already exists!')

Directory already exists!
Directory already exists!


Load the dataset and add split columns

In [4]:
adata = sc.read_h5ad('GSE154567_qc_hvg_anno_5k_raw_train_split.h5ad')

In [5]:
adata.obs['barcodes_orig'] = adata.obs.index.tolist()

In [6]:
metadata = pd.read_csv('../DatasetSplits/Metadata_Splits/GSE154567_metadata_splits.csv')

In [7]:
adata.obs = adata.obs.merge(metadata, left_on='barcodes_orig', right_on='index', copy=False, suffixes=('', '_drop'))
adata.obs = adata.obs[adata.obs.columns[~adata.obs.columns.str.endswith('_drop')]]
adata.obs.index = adata.obs['index']

/Users/odmini/miniforge3/envs/scrnaseq_torch/lib/python3.10/site-packages/anndata/_core/anndata.py:798: UserWarning: 
AnnData expects .obs.index to contain strings, but got values like:
    [0, 1, 2, 3, 4]

    Inferred to be: integer

  value_idx = self._prep_dim_index(value.index, attr)


### Run RandomForest on each split

In [8]:
splits = [f'Split_{i}' for i in range(1,6)]

In [9]:
def runSplitRF(adata, split, cluster_col='celltypes'):
    """
    Running RandomForest on multiple splits of the dataset
    """
    
    print(f"Running a Random Forest Classifier on split: {split}")
    
    # separating the data based on the split column
    adata_train = adata[adata.obs[split] == 'train']
    adata_test = adata[adata.obs[split] == 'test']
    
    # converting data from sparse to dense
    X_train = adata_train.raw.X.todense()
    X_test = adata_test.raw.X.todense()
    
    y_train = adata_train.obs[cluster_col].tolist()
    y_test = adata_test.obs[cluster_col].tolist()
    
    # Running the model and calculating runtime
    start_time = time.time()
    clf = RandomForestClassifier(random_state=2022, n_jobs=4)
    clf.fit(X_train, y_train)
    end_time = time.time()
    total_runtime = (end_time - start_time)
    print(f"RF Runtime: {total_runtime:2.2f}")
    
    # generating the predictions
    y_pred = clf.predict(X_test)
    
    # model evaluation
    acc = accuracy_score(y_test, y_pred)
    print(f'Accuracy: {acc:4.4f}')
    
    # calculating the precision/recall based multi-label F1 score
    macro_score = f1_score(y_test, y_pred, average = 'macro' )
    w_score = f1_score(y_test, y_pred,average = 'weighted' )
    print(f'    -> Non-Weighted F1 Score on validation set: {macro_score:4.4f} ' )
    print(f'    -> Weighted F1 Score on validation set: {w_score:4.4f} ' )
    print(class_rep(y_test,y_pred))
    
    
    return clf, total_runtime, acc, macro_score, w_score

### Running

In [10]:
model_eval = {}

In [11]:
for i in splits:
    
    print(f"Working on Split: {i}")
    
    # create a dict for storing current split information
    split_dict = {'Runtime':[], 
                  'Accuracy':[], 
                  'Macro_Score':[], 
                  'Weighted_Score':[]}
    clf, total_runtime, acc, macro_score, w_score = runSplitRF(adata, 
                                                               i, 
                                                               cluster_col='celltypes')
    
    joblib.dump(clf, f"./{model_dir}/{dataset_name}_{i}_RF.pkl")
    
    split_dict['Runtime'].append(total_runtime)
    split_dict['Accuracy'].append(acc)
    split_dict['Macro_Score'].append(macro_score)
    split_dict['Weighted_Score'].append(w_score)
    
    model_eval[i]=split_dict
    
    

Working on Split: Split_1
Running a Random Forest Classifier on split: Split_1


/Users/odmini/miniforge3/envs/scrnaseq_torch/lib/python3.10/site-packages/sklearn/utils/validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


RF Runtime: 16.83


/Users/odmini/miniforge3/envs/scrnaseq_torch/lib/python3.10/site-packages/sklearn/utils/validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


Accuracy: 0.9072
    -> Non-Weighted F1 Score on validation set: 0.8265 
    -> Weighted F1 Score on validation set: 0.9040 
                           precision    recall  f1-score   support

                  B cells       0.97      0.97      0.97      1937
              CD4 T cells       0.89      0.97      0.93      5064
              CD8 T cells       0.86      0.83      0.84      1852
                 NK cells       0.89      0.83      0.86       836
             erythrocytes       0.96      0.87      0.92      1044
         monocytes + cDCs       0.97      0.96      0.97      1212
             plasma cells       1.00      0.82      0.90        61
proliferating lymphocytes       1.00      0.19      0.32        80
 unidentified lymphocytes       0.82      0.67      0.73       888

                 accuracy                           0.91     12974
                macro avg       0.93      0.79      0.83     12974
             weighted avg       0.91      0.91      0.90     12974

W

...

Accuracy: 0.9123
    -> Non-Weighted F1 Score on validation set: 0.8233 
    -> Weighted F1 Score on validation set: 0.9091 
                           precision    recall  f1-score   support

                  B cells       0.97      0.98      0.97      1885
              CD4 T cells       0.89      0.97      0.93      5065
              CD8 T cells       0.86      0.84      0.85      1882
                 NK cells       0.92      0.85      0.88       881
             erythrocytes       0.98      0.89      0.93      1085
         monocytes + cDCs       0.97      0.97      0.97      1213
             plasma cells       1.00      0.74      0.85        58
proliferating lymphocytes       1.00      0.16      0.27        76
 unidentified lymphocytes       0.86      0.67      0.75       829

                 accuracy                           0.91     12974
                macro avg       0.94      0.78      0.82     12974
             weighted avg       0.91      0.91      0.91     12974

W

/Users/odmini/miniforge3/envs/scrnaseq_torch/lib/python3.10/site-packages/sklearn/utils/validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


RF Runtime: 16.91


/Users/odmini/miniforge3/envs/scrnaseq_torch/lib/python3.10/site-packages/sklearn/utils/validation.py:727: FutureWarning: np.matrix usage is deprecated in 1.0 and will raise a TypeError in 1.2. Please convert to a numpy array with np.asarray. For more information see: https://numpy.org/doc/stable/reference/generated/numpy.matrix.html
  warnings.warn(


Accuracy: 0.9102
    -> Non-Weighted F1 Score on validation set: 0.8267 
    -> Weighted F1 Score on validation set: 0.9075 
                           precision    recall  f1-score   support

                  B cells       0.96      0.98      0.97      1882
              CD4 T cells       0.89      0.97      0.93      5073
              CD8 T cells       0.87      0.83      0.85      1834
                 NK cells       0.89      0.84      0.87       818
             erythrocytes       0.97      0.88      0.92      1092
         monocytes + cDCs       0.97      0.96      0.96      1237
             plasma cells       1.00      0.79      0.88        63
proliferating lymphocytes       1.00      0.17      0.29        64
 unidentified lymphocytes       0.84      0.70      0.76       911

                 accuracy                           0.91     12974
                macro avg       0.93      0.79      0.83     12974
             weighted avg       0.91      0.91      0.91     12974



In [12]:
model_eval

{'Split_1': {'Runtime': [16.827188968658447],
  'Accuracy': [0.9071990134114383],
  'Macro_Score': [0.8265098481847232],
  'Weighted_Score': [0.9040163009295573]},
 'Split_2': {'Runtime': [16.64318013191223],
  'Accuracy': [0.9055033143209495],
  'Macro_Score': [0.8224796562322081],
  'Weighted_Score': [0.901920350984167]},
 'Split_3': {'Runtime': [17.00079107284546],
  'Accuracy': [0.9069677817172808],
  'Macro_Score': [0.8253684935457186],
  'Weighted_Score': [0.9035987378054597]},
 'Split_4': {'Runtime': [16.92686367034912],
  'Accuracy': [0.9122861106829042],
  'Macro_Score': [0.8233376820889956],
  'Weighted_Score': [0.9090976034067869]},
 'Split_5': {'Runtime': [16.91014313697815],
  'Accuracy': [0.9102050254354863],
  'Macro_Score': [0.8267490648867439],
  'Weighted_Score': [0.9074644144097459]}}

### Save the dictionary with metrics

In [13]:
# save the dictionary
def Pickler(data, filename):
    
    outfile = open(filename, 'wb+')
    
    #source destination
    
    pickle.dump(data, outfile)
    
    outfile.close()

Pickler(model_eval, filename=f"{dictionary_dir}/{dataset_name}_RF_EvalDict.pickle")

In [14]:
# to load

def Unpickler(filename):
    
    infile = open(filename, 'rb+')
    
    return_file = pickle.load(infile);
    
    infile.close()

    return return_file

test_loaddict = Unpickler(filename=f"{dictionary_dir}/{dataset_name}_RF_EvalDict.pickle")

In [15]:
test_loaddict

{'Split_1': {'Runtime': [16.827188968658447],
  'Accuracy': [0.9071990134114383],
  'Macro_Score': [0.8265098481847232],
  'Weighted_Score': [0.9040163009295573]},
 'Split_2': {'Runtime': [16.64318013191223],
  'Accuracy': [0.9055033143209495],
  'Macro_Score': [0.8224796562322081],
  'Weighted_Score': [0.901920350984167]},
 'Split_3': {'Runtime': [17.00079107284546],
  'Accuracy': [0.9069677817172808],
  'Macro_Score': [0.8253684935457186],
  'Weighted_Score': [0.9035987378054597]},
 'Split_4': {'Runtime': [16.92686367034912],
  'Accuracy': [0.9122861106829042],
  'Macro_Score': [0.8233376820889956],
  'Weighted_Score': [0.9090976034067869]},
 'Split_5': {'Runtime': [16.91014313697815],
  'Accuracy': [0.9102050254354863],
  'Macro_Score': [0.8267490648867439],
  'Weighted_Score': [0.9074644144097459]}}